In [ ]:
import re
import time
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyspark.sql.functions as F

from copy import copy
from typing import List
from datetime import datetime

from pyspark.ml.evaluation import Evaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import concat, col, lit, rand
from sklearn.metrics import (
    confusion_matrix,
    f1_score,
    roc_auc_score,
    roc_curve,
    accuracy_score,
    precision_score,
    r2_score,
    precision_recall_curve
)

## Global Constant

In [ ]:
KycDM: str = 'zt_dm_kyc_data'            # Схема KYC на чтение
arfsDM: str = 'zt_dm_aso_dfm_arfs'       # Схема ARFS, с правами на запись. Таблица создается сюда
target_table: str = 'tmp_target_table'   # таблица полученная из крансых и зеленых клиентов
    
filter_date: str = '2021-12-04'          # фильтр для fct_**
max_score: int = 1000                    # Максимальное значение риска
middle: int = 500                        # "Среднее" рисковое значение
uncertanity: int = 150                   # Значение окрестности для желтых, 0 - нет желтых

PATH: str = r'../FR/PythinProjects/ML_weights/'

ignored_typolygy: List[int] = [6, ]      # пропущенные типологии
ignored_columns: List[str] = [
    'fct_shady_risks',
]
ignored_scenatios: List[str] = [
    'R01.51 Белый список',
    'R01.52 Исполнители государственных контрактов',
    'R01.53 Участники внешнеэкономической деятельности',
]

In [ ]:
# get calendar id
try:
    calendar_id = spark.sql(f'select id from {KycDM}.dim_calendar where day = {filter_date}').collection()
except:
    calendar_id = None
print(f'Choose calendar_id: {calendar_id}')

# Visualition

In [ ]:
def plot_roc_curve(y_true, probs):
    fpr, tpr, _ = roc_curve(y_true, probs)
    auc = roc_auc_score(y_true, probs)
    
    plt.figure(figsize=[7, 6])
    plt.plot(fpr, tpr, color='darkorange', label=f'ROC curve (area = {auc:.2f}')
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.title('ROC-curve')
    plt.show()
    return None


def plot_pr_curve(y_true, probs):
    fpr, tpr, _ = precision_recall_curve(y_true, probs)
    
    plt.plot(fpr, tpr, color='darkorange')
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([0.0, 1.05])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.title('ROC-curve')
    plt.show()
    return None

# portfolio recovery

In [ ]:
def feature_dataframe(target_df, typology, verbose=False):
    table_names = list(set([risk['table_name'] for risk in typology['risks'] if not risk['table_name'] == 'fct_shady_risks']))
    total_rist: int = 0
    
    for table_name in table_names:
        risk_columns = [risk['column_name'] for risk in typology['risks'] if risk['table_name'] == table_name]
        total_risks += len(risk_columns)
        if verbose:
            print(f'{table_name}:\t\t\t{len(risk_columns)} columns')
        
        df = spark.sql(f'select * from {KycDM}.{table_name}')
        
        # Проверяем наличие всех необходимых сценариев в fct_ таблицах
        absent_columns = [r for r in risk_columns if not r in df.schema.names]
        
        if len(absent_columns) > 0:
            raise Exception(f'Cant find {";".join(absent_columns)} in {KycDM}.{table_name}')
        
        # Если объявлен calander_id, фильтруем таблицы
        if calendar_id and 'calendar_id' in df.scema.names:
            df = df.filter(f'calendar_id = {calendar_id}')
        
        # Проверяем пустая ли таблица
        if df.count() == 0:
            print(f'Warning\t\t{table_name} is empty')
        
        select_columns = [f'{column_name} as {table_name}__{column_name}' for column_name in risk_columns]
        
        select_columns = ['client_id', ] + select_columns
        
        df = df.selectExpr(*select_columns)
        
        if not target_df:
            target_df = df
            continue
        
        target_df = target_df.join(df, on=['client_id',], how='inner')
    
    if verbose:
        print('0'*50, f'\nTotal Columns: {total_risks}')
    return target_df

## Create DataSet

In [ ]:
def create_dataset(typology, return_assembler=False):
    print(f"{typology['dubious_id']} - {typology['description']}\n")
    
    # get only "красные"
    target_df = spark.sql(f'select client_id, label, from {arfsDM}.{target_table} where dubious_id = {typology["dubious_id"]}')
    
    # get onlu 'зеленые'
    greens = spark.sql(f'select client_id, table from {arfsDM}.{target_table} where label = 0')
    
    # balanced and join
    green_fraction = target_df.count() / greens.count() * 1.2
    target_df = target_df.union(greens.sample(green_fraction).limit(target_df.count()))
    
    # !
    df = feature_dataframe(target_df, typology, verbose=True)
    df = df.fillna(False)
    df = df.na.fill('false') # на сулчай пустых таблиц
    
    print('\t\t\tБаланс классов')
    df.groupby('label').count().show()
    
    # Transformation to Boolean
    print('Преобразование типов')
    for column in df.columns[2:]:
        df = df.withColumn(column, col(column).cast('Boolearn'))
        
    print('Сборка колончатых признаков')
    inputCols = copy(df.schema.names)
    inputCols.remove('label')
    inputCols.remove('client_id')
    
    # Объеденим все столбцы кроме 'label' & 'client_id'
    assembler = VectorAssembler().setInputCols(inputCols).setOutputCol('features')
    assembler_df = assembler.transform(df)
    
    try:
        print(f'Сохранение в {arfsDM}.tmp_train_test_dataset')
        assembler_df.write.mode('overwrite').saveAsTable(f'{arfsDM}.tmp_train_test_dataset')
        
        print('DONE')
        assembler_df = spark.sql(f'select * from {arfsDM}.tmp_train_test_dataset')
        
        if return_assembler:
            return assembler_df, assembler
        else:
            assembler_df
    except Exception as e:
        print(f"{typology['dubious_id']}: {e}")

# Custom class for Metrics

In [ ]:
class CustomEvaluator(Evaluator):
    # тк лэйбл кодируется 0 или 1000, делаем костомный фильтр
    label_true: int = 1000
    label_false: int = 0
    predict_middle: int = 500 # для придикта далем пороговое значение для фильтра > 650 - красный клиент
        
    def __init__(self, predictionCol=None, labelCol=None, metric=None) -> None:
        self.predictionCol = predictionCol
        self.labelCol = labelCol
        self.metric = metric
    
    def _evaluate(self, dataset) -> float:
        if self.metric == 'f1':
            return self.__f1(dataset)
        elif self.metric == 'accuracy':
            return self.__acuracy(dataset)
        elif self.metric == 'precision':
            return self.__precision(dataset)
        elif self.metric == 'recal':
            return self.__recal(dataset)
        else:
            print('Metric not aavalible\nAccuracy by default')
            return self.__acuracy(dataset)
    
    def isLargerBetter(self) -> bool:
        return True
    
    def __f1(self, dataset) -> float:
        tp = dataset.filter((F.col(self.labelCol) == self.label_true) & (F.col(self.predictionCol) > self.predict_middle)).count()
        fp = dataset.filter((F.col(self.labelCol) == self.label_false) & (F.col(self.predictionCol) < self.predict_middle)).count()
        tn = dataset.filter((F.col(self.labelCol) == self.label_true) & (F.col(self.predictionCol) < self.predict_middle)).count()
        return (2 * tp) / (2 * tp + fp + tn)
    
    def __accuracy(self, dataset) -> float:
        tp = dataset.filter((F.col(self.labelCol) == self.label_true) & (F.col(self.predictionCol) > self.predict_middle)).count()
        tn = dataset.filter((F.col(self.labelCol) == self.label_true) & (F.col(self.predictionCol) < self.predict_middle)).count()
        return (tp + tn) / dataset.count()
    
    def __precision(self, dataset) -> float:
        tp = dataset.filter((F.col(self.labelCol) == self.label_true) & (F.col(self.predictionCol) > self.predict_middle)).count()
        fp = dataset.filter((F.col(self.labelCol) == self.label_true) & (F.col(self.predictionCol) > self.predict_middle)).count()
        return tp / (tp + fp)
    
    def __recal(self, dataset) -> float:
        tp = dataset.filter((F.col(self.labelCol) == self.label_true) & (F.col(self.predictionCol) > self.predict_middle)).count()
        fn = dataset.filter((F.col(self.labelCol) == self.label_false) & (F.col(self.predictionCol) < self.predict_middle)).count()
        return tp / (tp + fn)
    

# Learn Models

In [ ]:
def create_model(train, num_folds: int = 5):
    '''
        input:
            - train - train dataset
            - num_folds - count folds
        output:
            - model with best parametrs
    '''
    
    pool: int = 6 # кол-во потоков
        
    estimator = LinearRegression(featuresCol='features', labelCol='label', fitIntercept=False)
    
    estimator_grid = ParamGridBuilder().\
        addGrid(estimator.regParam, np.linspace(0.01, 0.001, 10)).\
        addGrid(estimator.elasticnetParam, np.linspace(0, 1, 10)).\
        build()
    
    # Костомные метрики для регрессионной модели
    evalutor = CustomEvaluator(predictionCol='prediction', labelCol='label', metric='f1')
    
    print(f'Start Validation with: {num_folds} folds')
    start = time.time()
    
    # кроссвалидация с поиском по сетке
    cvModel = CrossValidator(
        estimator=estimator,
        estimatorParamMaps=estimator_grid,
        evaluator=evalutor,
        numFolds=num_folds,
        parallelism=pool,
    )
    model = cvModel.fit(train)

    print(f'\nTime crossval: {time.time() - start}')

    # освобождаем память
    del cvModel
    
    print(f'\nF1:\t{model.avgMetrics}')
    
    # get best params
    hyper_dict = dict()
    hyper_params = model.getEstimatorParamMaps()[np.argmax(model.avgMetrics)]
    
    for i in range(len(hyper_params.items())):
        hyper_name = re.search("name='(.+?)'", str([x for x in hyper_params.items()][i])).group(1)
        hyper_value = [x for x in hyper_params.items()][i][1]
        hyper_dict[hyper_name] = hyper_value
    
    print(f'\n\nBest model: {hyper_dict}\nMetric: {max(model.avgMetrics):.3f}')
    
    spark.catalog.clearCache()

    return model

# Test model

In [ ]:
def test_model(model, test, description:str=''):
    best_model = model.bestModel
    
    test_df = best_model.transform(test).select('label', 'prediction').toPandas()
    
    # features enginering
    test_dt['label'] = test_df['label'].apply(lambda x: 1 if x == 1000 else 0)
    test_dt['prediction_true'] = test_df['prediction'].apply(lambda x: 1000 if x > 1000 else x)
    test_dt['prediction_true'] = test_df['prediction_true'].apply(lambda x: 0 if x < 0 else x)
    test_dt['probability'] = test_df['prediction_true'] / max_score
    test_dt['prediction_true'] = test_df['prediction_true'].apply(lambda x: 1 if x >= 650 else 0)
    
    y_true = test_df['label']
    y_pred = test_df['prediction_true']
    y_proba = test_df['probability']
    
    # regression mertics
    rmse = model.bestModel.summary.rootMeanSquaredError
    r2 = model.bestModel.summary.r2
    
    # classsification metrics
    precision = precision(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    gini = 2 * auc_roc - 1
    conf_matrix = confusion_matrix(y_true, y_pred, labels=[1, 0])
    
    print('\n====Metrics on test dataset=====')
    print(f'\tF1:\t\t{f1:.3f}')
    print(f'\tAccuracy:\t{accuracy:.3f}')
    print(f'Precision:\t{precision:.3f}')
    print(f'AUC-ROC:\t{auc_roc:.3f}')
    print(f'Gini:\t\t{gini:.3f}')
    print(f'RMSE:\t\t{rmse:.3f}')
    print(f'r2:\t\t{r2:.3f}')
    print('*' * 40)
    print(f'Confusion matrix:\n{conf_matrix}')
    print('*' * 40)
    
    plot_roc_curve(y_true, y_proba, description)

## Get Typolygies

In [ ]:
risk_columns_query = f"\
    SELECT DISTINCT w.ID, w.DUBIOUS_ID, d.description as dubiuos_description, w.table_name, w.column_name, r.scenario_code, r.description as scenario_description \
    FROM {KycDM}.hw_dubious_weight as w \
    LEFT JOIN {KycDM}.dim_risk_columns as r \
    ON w.table_name = r.table_name and w.column_name = r.column_name \
    LEFT JOIN {KycDM}.dim_dubious as d \
    ON w.dubious_id = d.id \
    "
risks_columns = spark.sql(risk_columns_query)

#global list of Typologies
weights = []
dubious = risks_columns.select(col('dubious_id'), col('dubiuos_description')).distinct().collect()

for dub_id, description in dubious:
    typology = {
        "dubious_id": dub_id,
        "description": description,
        'medium_ppm': int(middle - uncertanity),
        'high_ppm': int(middle + uncertanity),
        'risks': [],
    }
    weights.append(typology)

# adding list of risks to each typology
for typology in weights:
    dubious_id = typology['dubious_id']
    for (risk_id, table_name, column_name, description) in risks_columns.filter(
        f"dubious_id = {dubious_id}").select("ID", 'table_name', 'column_name', 'scenario_description').collect() :
        
        # Выставляем максимальный риск для fct_shady_risks, т.к. это и есть спец. перечни. НЕ будет участвовать в обучении.
        weight = max_score if table_name == "fct_shady_risks" else 0
        typology['risks'].append(
            {
                'id': risk_id,
                'table_name': table_name,
                'column_name': column_name,
                'description': description, 
                'weight'     : weight,
            })
            
print(f"Всего типологий риска: {len(weights)}\n")
for t in weights:
    print(f"id{t['dubious_id']} - {t['description'][:30]}: {len(t['risks'])} сценариев")

## Learn models

In [ ]:
%matplotlib inline

for typology in weights:
    #пропускаем неиспользуемые типологии
    if typology['dubious_id'] in ignored_typology:
        continue

    dataset, assembler = create_dataset(typology, return_assembler=True)

    train, test = dataset.randomSplit([0.8, 0.2], seed=42)
        
    print("\nОбучение моделей...")
    model = create_model(train, num_folds=5)
    
    columns = test.columns[2:-1]
    
    print('=' * 28, 'Тестирование модели', '=' * 28)
    title_plot = f'{typology["dubious_id"]} - {typology["description"]}'
    test_model(model, test, title_plot)
    
    print(f'\n\tHyperParameters Statistic\n{get_stats(model)}')
    
    print('\nValue wieghts for each scenarios')
    weights_df = get_weights(model, columns)
    print(weights_df.head())
    
    print('\n\t Add weights to dict\n')
    for risk in typology['risks']:
        for ind, row in weights_df.iterrows():
            if risk['column_name'] in ignored_columns:
                continue
            if risk['table_name'] + '__' + risk['column_name'] == row['column']:
                rosk['weight'] = round(row['coef'], 2)
    del weights_df
    
    #clear cache
    spark.catalog.clearCache()
print('=' * 30, 'Done', '*' * 30)

# delete 'client_type'

In [ ]:
for weight in weights:
    weight.pop('client_type', None)
    for risk in weight['risks']:
        risk.pop('client_type', None)

## Load weights to excel-file

In [ ]:
lst = []

for row in weights:
    for risk in row['risks']:
        dct = {}
        dct['dubious_id'] = row['dubious_id']
        dct['typology'] = row['typology']
        dct['description'] = risk['description']
        dct['weight'] = risk['weight']
        lst.append(dct)

pd.DataFrame(lst).to_excel(f'{PATH}/typologies_{filter_date}.xlsx', index=False)